In [1]:
import pandas as pd
import ipaddress
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout


In [3]:
import numpy as np

df = pd.read_csv("Darknet.csv")

# Drop unnecessary columns
df = df.drop(["Flow ID", "Timestamp", "Label2"], axis=1)

# Convert IP addresses to integers safely
df['Src IP'] = df['Src IP'].apply(lambda x: int(ipaddress.ip_address(x)))
df['Dst IP'] = df['Dst IP'].apply(lambda x: int(ipaddress.ip_address(x)))

# Encode labels
label_encoder = LabelEncoder()
df['Label1'] = label_encoder.fit_transform(df['Label1'])

# Replace inf and -inf with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN
df = df.dropna()

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop("Label1", axis=1))

df_processed = pd.DataFrame(X_scaled, columns=df.drop("Label1", axis=1).columns)
df_processed["Label1"] = df["Label1"]

df_processed.head()


,Src IP,Src Port,Dst IP,Dst Port,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label1
0,-0.523268,0.976241,1.490986,-0.768559,-0.78203,-0.543455,-0.061661,-0.043666,-0.032777,-0.027695,...,0.591618,0.0,0.0,0.0,0.0,-1.011921,-0.292716,-1.026108,-0.843165,0.0
1,-0.523268,0.976293,1.490986,-0.768559,-0.78203,-0.543450,-0.061661,-0.043666,-0.032777,-0.027695,...,0.591618,0.0,0.0,0.0,0.0,-1.011921,-0.292716,-1.026108,-0.843165,0.0
2,-0.523268,0.976345,1.490986,-0.768559,-0.78203,-0.543449,-0.061661,-0.043666,-0.032777,-0.027695,...,0.591618,0.0,0.0,0.0,0.0,-1.011921,-0.292716,-1.026108,-0.843165,0.0
3,-0.523268,0.559405,-0.404452,-0.768559,-0.78203,-0.543451,-0.061661,-0.043666,-0.032777,-0.027695,...,0.591618,0.0,0.0,0.0,0.0,-1.011921,-0.292716,-1.026108,-0.843165,0.0
4,-0.523268,-0.190577,0.923033,0.084572,-0.78203,-0.259827,0.200769,0.079828,-0.011771,-0.026152,...,0.591618,0.0,0.0,0.0,0.0,1.026766,-0.292716,0.957368,1.168882,0.0


In [5]:
import numpy as np

# Remove any remaining inf or NaN values
df_processed.replace([np.inf, -np.inf], np.nan, inplace=True)
df_processed = df_processed.dropna()

# Reset index after dropping rows
df_processed = df_processed.reset_index(drop=True)

# Check for NaN
print("NaN count:\n", df_processed.isna().sum().sum())


NaN count:
 0


In [6]:
from sklearn.model_selection import train_test_split

X = df_processed.drop("Label1", axis=1)
y = df_processed["Label1"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (126812, 81)
Test shape: (31704, 81)


In [7]:
import tensorflow as tf

baseline_model = tf.keras.Sequential([
    tf.keras.layers.Dense(250, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

baseline_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history1 = baseline_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=512,
    validation_data=(X_test, y_test),
    verbose=1
)

loss1, acc1 = baseline_model.evaluate(X_test, y_test, verbose=0)
print("✅ Baseline Accuracy:", acc1)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9070 - loss: 0.3103 - val_accuracy: 0.9860 - val_loss: 0.0549
Epoch 2/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9875 - loss: 0.0498 - val_accuracy: 0.9894 - val_loss: 0.0431
Epoch 3/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9906 - loss: 0.0399 - val_accuracy: 0.9902 - val_loss: 0.0395
Epoch 4/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9907 - loss: 0.0378 - val_accuracy: 0.9898 - val_loss: 0.0398
Epoch 5/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9913 - loss: 0.0336 - val_accuracy: 0.9910 - val_loss: 0.0354
Epoch 6/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9915 - loss: 0.0326 - val_accuracy: 0.9907 - val_loss: 0.0363
Epoch 7/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9919 - loss: 0.0297 - val_accuracy: 0.9903 - val_loss: 0.0350
Epoch 8/20
248/248 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9919 - loss: 0.0291 - val_accuracy

In [8]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

improved_model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu'),
    Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

improved_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history2 = improved_model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=512,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

loss2, acc2 = improved_model.evaluate(X_test, y_test, verbose=0)
print("🚀 Improved Accuracy:", acc2)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.8768 - loss: 0.3915 - val_accuracy: 0.9831 - val_loss: 0.0633
Epoch 2/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9807 - loss: 0.0763 - val_accuracy: 0.9877 - val_loss: 0.0480
Epoch 3/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.9855 - loss: 0.0559 - val_accuracy: 0.9886 - val_loss: 0.0449
Epoch 4/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9870 - loss: 0.0509 - val_accuracy: 0.9892 - val_loss: 0.0418
Epoch 5/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9883 - loss: 0.0446 - val_accuracy: 0.9896 - val_loss: 0.0415
Epoch 6/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9884 - loss: 0.0445 - val_accuracy: 0.9901 - val_loss: 0.0422
Epoch 7/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9893 - loss: 0.0423 - val_accuracy: 0.9906 - val_loss: 0.0379
Epoch 8/50
248/248 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9893 - loss: 0.0412 - val_acc

In [9]:
print("Baseline Accuracy:", acc1)
print("Improved Accuracy:", acc2)
print("Accuracy Improvement:", acc2 - acc1)


Baseline Accuracy: 0.9927769303321838
Improved Accuracy: 0.9924299716949463
Accuracy Improvement: -0.00034695863723754883
